In [ ]:
import cv2
import imutils
import numpy as np
import pytesseract
from ipynb.fs.full.ColourSpaceConversionUtil import *


def getLicensePlateNumberFromFrame(frameFilePath):
    
    # Constants for pytesseract
    PYTESSERACT_APPLICATION_FILE_PATH=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    # Setting the pytesseract application file path
    pytesseract.pytesseract.tesseract_cmd = PYTESSERACT_APPLICATION_FILE_PATH
    
    # Read the frame
    frame = cv2.imread(frameFilePath, cv2.COLOR_BGR2RGB)
    
    # Resize the frame
    frame = cv2.resize(frame,(600,400))
    
    # Convert the frame from RGB to grayscale
    grayscaleFrame=colourRGBToGrayConversion(frame)
    
    # Remove the unwanted noise from the grayscale frame using a bilateral filter
    # Arg 1 is the source image on which the bilateral filter is to be applied
    # Arg 2 is the diameter of the pixels
    # Arg 3 is the sigma colour
    # Arg 4 is the sigma space
    grayscaleFrame = cv2.bilateralFilter(grayscaleFrame, 13, 15, 18)
    
    # Perform edge Detection using Canny Edge Detector
    # Arg1 is the grayscale image 
    # Arg2 is the first threshold
    # Arg3 is the second threshold
    # Arg4 is the aperture size
    edgesInTheGrayscaleFrame = cv2.Canny(grayscaleFrame, 30, 200)
    
    # To find the outlines of all the detected edges in the binary image of the grayscale image
    # Arg 1 is the source image
    # Arg 2 is the contour retrieval mode
    # Arg 3 is to remove all the points on the edges of the contour and compresses it
    allContoursInTheFrame = cv2.findContours(edgesInTheGrayscaleFrame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Returns the counters of the countours
    allContoursInTheFrame = imutils.grab_contours(allContoursInTheFrame)
    
    # Sort all the obtain countours based on its area and retain only the top 10
    allContoursInTheFrame = sorted(allContoursInTheFrame, key = cv2.contourArea, reverse = True)[:10]
    
    # A variable that stores the contours of the license plate
    licensePlateContour = None
    
    for eachCountour in allContoursInTheFrame:
        
        # Calculate the perimeter of the closed countour
        countourPerimeter = cv2.arcLength(eachCountour, True)
        
        # Approximate the contour shape to another shape
        epsilon = 0.018 * countourPerimeter
        approximateContourShape = cv2.approxPolyDP(eachCountour, epsilon, True)
 
        # Check to see if the approximate shape of the contour is a rectangle or square with four points
        if len(approximateContourShape) == 4:
            licensePlateContour = approximateContourShape
            break

    # If the license plate is detected
    if licensePlateContour is not None:
        
        # Draw the contour on the frame to identify the location of the license plate
        cv2.drawContours(frame, [licensePlateContour], -1, (0, 0, 255), 3)
        
        # Masking the rest of the image apart from the identified contour
        mask = np.zeros(grayscaleFrame.shape,np.uint8)
        maskedImageWithContour = cv2.drawContours(mask,[licensePlateContour],0,255,-1,)
        maskedImageWithContour = cv2.bitwise_and(frame,frame,mask=mask)
        
        # Cropping the license plate from the grayscale image
        (x, y) = np.where(mask == 255)
        (topX, topY) = (np.min(x), np.min(y))
        (bottomX, bottomY) = (np.max(x), np.max(y))
        licensePlateFromTheFrame = grayscaleFrame[topX:bottomX+1, topY:bottomY+1]
        
        # Save the license plate as an image
        cv2.imwrite('../file_backup/Red_Light_Violation_License_Plate.jpeg', licensePlateFromTheFrame)
        
        # Detecting the license plate number
        licensePlateNumber = pytesseract.image_to_string(licensePlateFromTheFrame, config='--psm 11')
        
        # Display the license plate number
        print('LicencePlateDetection ---- The red light violator\'s license plate number is ',licensePlateNumber)
  
    else:
        
        # If the license plate is NOT detected
        print ('LicencePlateDetection ---- The red light violator\'s license plate number could not be detected!')

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return;